In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

In [31]:
from src.wess_zumino_model import WessZuminoModel
from src.matrix_to_ps import matrix_to_pse
from src.binary_encodings import standard_encode
from src.qiskit_utilities import pauli_dict_to_op


from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import RealAmplitudes
from qiskit import BasicAer
from qiskit import qpy
from qiskit import QuantumCircuit, transpile


import scipy.sparse.linalg
import sympy as sp
import numpy as np

import math

In [32]:
def potential(self, n):
    # m*q with m=1
    return -self.mass*self.qs[n]

In [33]:
wz=WessZuminoModel(2,1.0,potential,'dirichlet')

In [34]:
for cutoff in [2,4,8]:
    #CLASSICAL COMPUTATION OF GROUND STATE ENERGY
    wz.construct_ham_matrix(cutoff)
    ens=scipy.sparse.linalg.eigs(wz.hamMat,k=6,sigma=0.0)[0]
    print(ens)

    #VQE COMPUTATION OF GROUND STATE ENERGY
    ps=matrix_to_pse(wz.hamMat, standard_encode)
    op=pauli_dict_to_op(ps.to_dict())


    nq=math.floor(math.log2(wz.hamMat.shape[0]))
    if not math.log2(wz.hamMat.shape[0]).is_integer():
        nq+=1

    ansatz = RealAmplitudes(nq, reps=cutoff)    
    backend = BasicAer.get_backend("statevector_simulator")
    vqe = VQE(ansatz=ansatz, optimizer=COBYLA(), quantum_instance=backend)
    results=[]
    for i in range(10):
        results.append(vqe.compute_minimum_eigenvalue(op).eigenvalue.real)
    en = np.array(results).min()
    print(en)

[0.00696601-1.1669081e-10j 0.00696601-7.0731360e-13j
 0.00696601+9.2554658e-11j 0.00696601+2.6414609e-10j
 1.1249988 +1.4144433e-08j 1.1249999 -1.7680571e-09j]
0.006966012533328223
[3.2249103e-05-2.3024865e-12j 9.7540504e-01-7.0885939e-08j
 9.7540593e-01-6.2025309e-09j 1.1180669e+00+9.8994440e-07j
 1.1180661e+00+1.8627521e-08j 1.2815913e+00+3.0211087e-08j]
0.000224516906106631
[1.0418004e-09-4.4621295e-17j 1.1386600e+00+3.7501209e-08j
 1.1180317e+00+2.0954742e-08j 1.1180329e+00-1.6845336e-08j
 1.1180340e+00-1.2223609e-08j 2.2360659e+00+2.7939626e-08j]
0.15672842655487607


In [35]:
for cutoff in [16,32]:
    #CLASSICAL COMPUTATION OF GROUND STATE ENERGY
    wz.construct_ham_matrix(cutoff)
    ens=scipy.sparse.linalg.eigs(wz.hamMat,k=6,sigma=0.0)[0]
    print(ens)

    #VQE COMPUTATION OF GROUND STATE ENERGY
    ps=matrix_to_pse(wz.hamMat, standard_encode)
    op=pauli_dict_to_op(ps.to_dict())


    nq=math.floor(math.log2(wz.hamMat.shape[0]))
    if not math.log2(wz.hamMat.shape[0]).is_integer():
        nq+=1

    ansatz = RealAmplitudes(nq, reps=cutoff)    
    backend = BasicAer.get_backend("statevector_simulator")
    vqe = VQE(ansatz=ansatz, optimizer=COBYLA(), quantum_instance=backend)
    results=[]
    for i in range(10):
        results.append(vqe.compute_minimum_eigenvalue(op).eigenvalue.real)
    en = np.array(results).min()
    print(en)

[5.3902821e-10-4.2777364e-17j 1.1225426e+00-3.0984197e-08j
 1.1180327e+00+1.6197318e-09j 1.1180334e+00+5.6862546e-09j
 1.1180334e+00-6.4749694e-09j 2.2360625e+00+5.3210269e-06j]
